<a href="https://colab.research.google.com/github/satsaras/datascience/blob/master/Apache%20pyspark%20classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kaggle

In [ ]:
from google.colab import files

In [ ]:
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"satsaras","key":"5553bfcf9fb38728203bc21cf93441be"}'}

In [ ]:
!rm -r ~/.kaggle
!mkdir ~/.kaggle
!mv ./kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets list

rm: cannot remove '/root/.kaggle': No such file or directory
ref                                                         title                                              size  lastUpdated          downloadCount  
----------------------------------------------------------  ------------------------------------------------  -----  -------------------  -------------  
gpreda/reddit-vaccine-myths                                 Reddit Vaccine Myths                              234KB  2021-08-07 09:10:20          10359  
crowww/a-large-scale-fish-dataset                           A Large Scale Fish Dataset                          3GB  2021-04-28 17:03:01           6336  
imsparsh/musicnet-dataset                                   MusicNet Dataset                                   22GB  2021-02-18 14:12:19           2373  
dhruvildave/wikibooks-dataset                               Wikibooks Dataset                                   2GB  2021-07-03 18:37:20           2590  
promptcloud/car

Read Data from kaggle

In [ ]:
!kaggle datasets download -d kritanjalijain/amazon-reviews

 99% 1.29G/1.29G [00:10<00:00, 107MB/s] 
100% 1.29G/1.29G [00:10<00:00, 133MB/s]


In [ ]:
! unzip /content/amazon-reviews.zip -d /content/

Archive:  /content/amazon-reviews.zip
replace /content/amazon_review_polarity_csv.tgz? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
!pip install pyspark

     |████████████████████████████████| 212.4 MB 68 kB/s 
     |████████████████████████████████| 198 kB 27.8 MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.2-py2.py3-none-any.whl size=212880768 sha256=c1d10a307c7e019d98b45d6fc11f46aab5ffe6edc26a4ca765072bee4c72d8c9
  Stored in directory: /root/.cache/pip/wheels/a5/0a/c1/9561f6fecb759579a7d863dcd846daaa95f598744e71b02c77
Successfully built pyspark


In [ ]:
import pyspark
from pyspark import SparkContext,SparkConf
from pyspark.sql import SparkSession
from pyspark.ml.feature import HashingTF, IDF, Tokenizer,CountVectorizer
from pyspark.ml.feature import Word2Vec
import pyspark.sql.functions as F
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression,RandomForestClassifier, GBTClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.feature import HashingTF, Tokenizer
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
import pandas as pd
from pyspark.ml.feature import Word2Vec
from pyspark.sql.functions import when
from pyspark.sql.types import *


Initiate Sparksession

In [ ]:
spark=SparkSession.builder.appName('test').getOrCreate()

Read data

In [ ]:
df=spark.read.csv('/content/train.csv')

Reanme Columns

In [ ]:
df=df.withColumnRenamed('_c0','Polarity')
df=df.withColumnRenamed('_c1','Title')
df=df.withColumnRenamed('_c2','Review')

Filter out small reviews as they will not provide enough signal

In [ ]:
df=df.withColumn("len_Review", F.length("Review"))
df=df.filter(df['len_Review']>10)

Add custom stop words(that are high indicator of polarity) to increase generalization capability of model

In [ ]:
custom_stopwords=['good','great','bad','hate','like','better','recommend','best','disappointed','perfect','return','easy','hard','ha','wa','love','problem','worst','boring','wrong','loved','wonderful','amazing']

In [ ]:
generic_stopwords=['all', 'six', 'just', 'less', 'being', 'indeed', 'over', 'move', 'anyway', 'four', 'not', 'own', 'through',
    'using', 'fifty', 'where', 'mill', 'only', 'find', 'before', 'one', 'whose', 'system', 'how', 'somewhere',
    'much', 'thick', 'show', 'had', 'enough', 'should', 'to', 'must', 'whom', 'seeming', 'yourselves', 'under',
    'ours', 'two', 'has', 'might', 'thereafter', 'latterly', 'do', 'them', 'his', 'around', 'than', 'get', 'very',
    'de', 'none', 'cannot', 'every', 'un', 'they', 'front', 'during', 'thus', 'now', 'him', 'nor', 'name', 'regarding',
    'several', 'hereafter', 'did', 'always', 'who', 'didn', 'whither', 'this', 'someone', 'either', 'each', 'become',
    'thereupon', 'sometime', 'side', 'towards', 'therein', 'twelve', 'because', 'often', 'ten', 'our', 'doing', 'km',
    'eg', 'some', 'back', 'used', 'up', 'go', 'namely', 'computer', 'are', 'further', 'beyond', 'ourselves', 'yet',
    'out', 'even', 'will', 'what', 'still', 'for', 'bottom', 'mine', 'since', 'please', 'forty', 'per', 'its',
    'everything', 'behind', 'does', 'various', 'above', 'between', 'it', 'neither', 'seemed', 'ever', 'across', 'she',
    'somehow', 'be', 'we', 'full', 'never', 'sixty', 'however', 'here', 'otherwise', 'were', 'whereupon', 'nowhere',
    'although', 'found', 'alone', 're', 'along', 'quite', 'fifteen', 'by', 'both', 'about', 'last', 'would',
    'anything', 'via', 'many', 'could', 'thence', 'put', 'against', 'keep', 'etc', 'amount', 'became', 'ltd', 'hence',
    'onto', 'or', 'con', 'among', 'already', 'co', 'afterwards', 'formerly', 'within', 'seems', 'into', 'others',
    'while', 'whatever', 'except', 'down', 'hers', 'everyone', 'done', 'least', 'another', 'whoever', 'moreover',
    'couldnt', 'throughout', 'anyhow', 'yourself', 'three', 'from', 'her', 'few', 'together', 'top', 'there', 'due',
    'been', 'next', 'anyone', 'eleven', 'cry', 'call', 'therefore', 'interest', 'then', 'thru', 'themselves',
    'hundred', 'really', 'sincere', 'empty', 'more', 'himself', 'elsewhere', 'mostly', 'on', 'fire', 'am', 'becoming',
    'hereby', 'amongst', 'else', 'part', 'everywhere', 'too', 'kg', 'herself', 'former', 'those', 'he', 'me', 'myself',
    'made', 'twenty', 'these', 'was', 'bill', 'cant', 'us', 'until', 'besides', 'nevertheless', 'below', 'anywhere',
    'nine', 'can', 'whether', 'of', 'your', 'toward', 'my', 'say', 'something', 'and', 'whereafter', 'whenever',
    'give', 'almost', 'wherever', 'is', 'describe', 'beforehand', 'herein', 'doesn', 'an', 'as', 'itself', 'at',
    'have', 'in', 'seem', 'whence', 'ie', 'any', 'fill', 'again', 'hasnt', 'inc', 'thereby', 'thin', 'no', 'perhaps',
    'latter', 'meanwhile', 'when', 'detail', 'same', 'wherein', 'beside', 'also', 'that', 'other', 'take', 'which',
    'becomes', 'you', 'if', 'nobody', 'unless', 'whereas', 'see', 'though', 'may', 'after', 'upon', 'most', 'hereupon',
    'eight', 'but', 'serious', 'nothing', 'such', 'why', 'off', 'a', 'don', 'whereby', 'third', 'i', 'whole', 'noone',
    'sometimes', 'well', 'amoungst', 'yours', 'their', 'rather', 'without', 'so', 'five', 'the', 'first', 'with',
    'make', 'once']

In [ ]:
df=df.rdd.takeSample(False, 100000, seed=0)

In [ ]:
df_sample=spark.createDataFrame(pd.DataFrame(df).rename({0:'Polarity',1:'Title',2:'Review'},axis=1).drop(3,axis=1))

In [ ]:
import gensim.parsing.preprocessing as preprocess
from gensim.parsing.preprocessing import STOPWORDS
from pyspark.sql.types import StringType
from gensim import utils
from gensim.utils import lemmatize
from textblob import TextBlob, Word

In [ ]:
my_stop_words = STOPWORDS.union(custom_stopwords)

In [ ]:
def remove_stopwords(s):
    s = utils.to_unicode(s)
    return " ".join(w for w in s.split() if w not in my_stop_words)

In [ ]:
def lemmetizer(s):
    s=TextBlob(s)
    return " ". join([w.lemmatize() for w in s.words])

In [ ]:
Pre_filters=[preprocess.strip_tags,
        preprocess.strip_punctuation,
        preprocess.strip_multiple_whitespaces,
        preprocess.strip_numeric,
        preprocess.strip_short,
        lemmetizer,
        remove_stopwords]

In [ ]:
def pre_process(x):
    review=x[2]
    review=review.lower()
    for f in Pre_filters:
        review=f(review)
    return(x[0],review)

In [ ]:
import nltk
nltk.download('punkt')
!python -m textblob.download_corpora

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package conll2000 to /root/nltk_data...
[nltk_data]   Package conll2000 is already up-to-date!
[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!
Finished.


In [ ]:
df_sample=df_sample.rdd.map(lambda x: pre_process(x))

In [ ]:
df_sample = df_sample.toDF(['Polarity','Review'])

In [ ]:
df_sample.withColumn('word', F.explode(F.split(F.col('Review'), ' '))) \
  .groupBy('word') \
  .count() \
  .sort('count', ascending=False) \
  .take(200)

[Row(word='book', count=104021),
 Row(word='time', count=40300),
 Row(word='movie', count=33085),
 Row(word='read', count=32982),
 Row(word='work', count=26661),
 Row(word='year', count=23087),
 Row(word='product', count=22652),
 Row(word='use', count=20336),
 Row(word='story', count=19954),
 Row(word='buy', count=19599),
 Row(word='bought', count=19586),
 Row(word='album', count=17948),
 Row(word='thing', count=17894),
 Row(word='way', count=17842),
 Row(word='new', count=16855),
 Row(word='song', count=16732),
 Row(word='know', count=16440),
 Row(word='think', count=16254),
 Row(word='little', count=16213),
 Row(word='doe', count=15805),
 Row(word='got', count=15588),
 Row(word='want', count=14878),
 Row(word='game', count=14689),
 Row(word='music', count=14563),
 Row(word='old', count=14184),
 Row(word='day', count=13987),
 Row(word='money', count=13825),
 Row(word='people', count=13794),
 Row(word='character', count=13729),
 Row(word='dvd', count=13078),
 Row(word='look', count=127

In [ ]:
def Tokenize(df):
    tokenizer = Tokenizer(inputCol="Review", outputCol="Review_Words")
    wordsData = tokenizer.transform(df)
    return(wordsData)


In [ ]:
def TFIDF(df):
    wordsData=Tokenize(df)
    hashingTF = HashingTF(inputCol="Review_Words", outputCol="rawFeatures", numFeatures=512)
    featurizedData = hashingTF.transform(wordsData)

    idf = IDF(inputCol="rawFeatures", outputCol="Review_CV")
    IDFModel = idf.fit(featurizedData)
    IDF_Data = IDFModel.transform(featurizedData)
    return(IDF_Data)


In [ ]:
IDF_Data=TFIDF(df_sample)

In [ ]:
def LogictiveRegression_CV(df_train,df_val):
    lr=LogisticRegression(labelCol='label',featuresCol='Review_CV')
    pipeline = Pipeline(stages=[lr])
    paramGrid = ParamGridBuilder() \
    .addGrid(lr.elasticNetParam, [0,0.2,0.7]) \
    .addGrid(lr.regParam, [0.1, 0.01,0.05]) \
    .build()
    eval=BinaryClassificationEvaluator(labelCol='label')
    crossval = CrossValidator(estimator=pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=BinaryClassificationEvaluator(),numFolds=2) 
    cv=crossval.fit(df_train)
    best_model=cv.bestModel.stages[0]
    prediction=best_model.transform(df_val)
    acc=eval.evaluate(prediction)
    print('The accuracy is %g' %acc)

In [ ]:

df_train.columns

['label', 'Review', 'Review_Words', 'rawFeatures', 'Review_CV']

In [ ]:
def train_test_Split(df):
    df = df.withColumnRenamed('Polarity','label')
    df = df.withColumn("label", df["label"].cast(IntegerType()))
    df = df.withColumn("label",when(df["label"] == 2, 0).otherwise(df["label"]))
    df = df.withColumn("label", df["label"].cast(IntegerType()))
    df_train,df_test=df.randomSplit([0.8,0.2])
    return(df_train,df_test)


In [ ]:
df_train,df_test=train_test_Split(word2vec_TF)

In [ ]:
LogictiveRegression_CV(df_train,df_test)

The accuracy is 0.744392


In [ ]:
LogictiveRegression_CV(df_train,df_test)

The accuracy is 0.836681


In [ ]:
def RandomForest_CV(df_train,df_val):
    rf=RandomForestClassifier(labelCol='label',featuresCol='Review_CV')
    pipeline = Pipeline(stages=[rf])
    paramGrid = ParamGridBuilder() \
    .addGrid(rf.maxDepth, [10,20]) \
    .addGrid(rf.numTrees, [50,100,150]) \
    .addGrid(rf.impurity ,['gini','entropy']) \
    .addGrid(rf.maxBins,[20,40]) \
    .build()
    eval=BinaryClassificationEvaluator(labelCol='label')
    crossval = CrossValidator(estimator=pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=BinaryClassificationEvaluator(),numFolds=2) 
    cv=crossval.fit(df_train)
    best_model=cv.bestModel.stages[0]
    prediction=best_model.transform(df_val)
    acc=eval.evaluate(prediction)
    print('The accuracy is %g' %acc)


In [ ]:
RandomForest_CV(df_train,df_test)

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:37719)
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/py4j/java_gateway.py", line 977, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/py4j/java_gateway.py", line 1115, in start
    self.socket.connect((self.address, self.port))
ConnectionRefusedError: [Errno 111] Connection refused


Py4JNetworkError: ignored

In [ ]:
RandomForest_CV(df_train,df_test)

Py4JJavaError: ignored

----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 38916)
ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/py4j/java_gateway.py", line 1207, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/py4j/java_gateway.py", line 1033, in send_command
    response = connection.send_command(command)
  File "/usr/local/lib/python3.7/dist-packages/py4j/java_gateway.py", line 1212, in send_command
    "Error while receiving", e, proto.ERROR_ON_RECEIVE)
py4j.protocol.Py4JNetworkError: Error while receiving
ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/py

In [ ]:
def GBTClassifier_CV(df_train,df_val):
    gbt=GBTClassifier(labelCol='label',featuresCol='Review_CV')
    pipeline = Pipeline(stages=[gbt])
    paramGrid = ParamGridBuilder() \
    .addGrid(gbt.maxDepth, [10,20]) \
    .addGrid(gbt.maxBins,[20,40]) \
    .addGrid(gbt.subsamplingRate,[0.3,0.4,0.5]) \
    .build()
    eval=BinaryClassificationEvaluator(labelCol='label')
    crossval = CrossValidator(estimator=pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=BinaryClassificationEvaluator(),numFolds=2) 
    cv=crossval.fit(df_train)
    best_model=cv.bestModel.stages[0]
    prediction=best_model.transform(df_val)
    acc=eval.evaluate(prediction)
    print('The accuracy is %g' %acc)


In [ ]:
GBTClassifier_CV(df_train,df_test)

KeyboardInterrupt: ignored

In [ ]:
def wor2vec(df):
    wordsData=Tokenize(df)
    w2v = Word2Vec(vectorSize=100, minCount=0, inputCol="Review_Words", outputCol="Review_CV")
    w2v_model = w2v.fit(wordsData)
    word2vec_TF = w2v_model.transform(wordsData)
    return(word2vec_TF)

In [ ]:
word2vec_TF=wor2vec(df_sample)

In [ ]:
word2vec_TF=word2vec_TF.withColumnRenamed('w2v_Features','Review_CV')

In [ ]:
word2vec_TF.printSchema()

root
 |-- Polarity: string (nullable = true)
 |-- Review: string (nullable = true)
 |-- Review_Words: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- w2v_Features: vector (nullable = true)

